Resumen

En este trabajo, se tomará un dataset de imágenes de galaxias y agujeros negros y en base a la arquitectura VGG-16 se establecerá un modelo que pueda clasificar imágenes entre ambos tipos

In [1]:
#Importando Módulos

import cv2
import keras
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
import scipy
import streamlit as st
from streamlit_jupyter import StreamlitPatcher, tqdm
from keras_visualizer import visualizer

El primer paso, luego de importar los módulos es el de crear, mediante una función que viene con Keras, un array de tensores para cada imagen y que los divide automáticamente según su tipo, y haciendo lo mismo para los datos del Train y Test, datos que fueron separados manualmente

In [2]:
#Estableciendo una seed random

tf.random.set_seed(0)

test_directory = 'test'

train_directory = 'train'

#Reescalando e importando el dataset de imágen, en este caso para el train

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
train_directory, 
target_size=(224,224), 
color_mode='rgb', 
batch_size=64, 
class_mode='binary', 
subset='training',
shuffle=True,
seed=42
)

#Mismo procedimiento, pero para los datos de Test (Fueron separados manualmente)

test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
test_directory,
target_size=(224,224), 
color_mode='rgb', 
batch_size=64, 
class_mode='binary', 
shuffle=False
)

Found 322 images belonging to 2 classes.
Found 78 images belonging to 2 classes.


El siguiente paso, muy importante, es el de construir el modelo y su arquitectura. Este modelo 

In [4]:
#-------------------------Arquitectura VGC-16---------------------------------

model = Sequential()

#Primer bloque de la arquitectura

model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

#Segundo bloque

model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

#Tercer bloque

model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

#Cuarto bloque

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

#Quinto bloque

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

#Agregando capas al modelo

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=2, activation="softmax"))

opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=['accuracy'])

#Parámetros del modelo

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 conv2d_14 (Conv2D)          (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 112, 112, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_15 (Conv2D)          (None, 112, 112, 128)     73856     
                                                                 
 conv2d_16 (Conv2D)          (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 56, 56, 128)       0         
 g2D)                                                 

ValueError: [Keras Visualizer] Error while visualizing: <class 'Exception'>

Ahora, el siguiente paso es entrenar el modelo con los datos que tenemos, pero también, va a ser necesario guardar los valores de los pesos. Se utilizaron nada más apenas 10 epochs por un tema de eficiencia, puesto que usar más epochs tardaría demasiado tiempo.

In [21]:
#-------------------------Entrenando el modelo--------------------------------

#Guardando variables del modelo

checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq=1)

#La siguiente función me permite parar el modelo si ve que no hay progreso luego de 5 epochs

early = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

#Entrenando el modelo en sí

model.fit(train_generator, batch_size= 100, epochs= 10, validation_data= test_generator, steps_per_epoch= 4, validation_steps=4, 
callbacks=[early], verbose=1)

#Guardando los pesos

model.save_weights('Weights')

Epoch 1/10


Por último, integramos Streamlit para abrir una terminal que nos permita subir una foto y que el modelo decida si es una galaxia o un agujero negro

In [17]:
np.random.seed(0)

st.title('Adiviná si es un agujero negro o una galaxia :galaxia: :agujero:')

uploaded_file = st.file_uploader('Ingrese una imagen, solo soporta png y jpg ', 
                type=['png', 'jpg'], 
                accept_multiple_files=False,
                key=None
                )


if uploaded_file is not None: 
    
    file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
    opencv_image = cv2.imdecode(file_bytes, 1)
    imagest.image(uploaded_file, channels="BGR")
    Image_ToPredict = ImageDataGenerator(opencv_image, 
    target_size=(224,224), 
    color_mode='rgb', 
    batch_size=64, 
    class_mode='binary', 
    shuffle=False)
    
    #Hacer que el modelo prediga que es
    
    predictions = model.predict(Image.ToPredict)
    classes = np.argmax(predictions, axis=1)